In [1]:
%load_ext autoreload
%autoreload 2

from model import *



from torch import empty, cat, arange
from torch.nn.functional import fold, unfold
from torch import nn

In [2]:
x = torch.randn((1, 3, 4, 4))

# x = torch.tensor(
    # [[0, 1., 0],     
    # [2, 0, 0],
    # [0, 0, 1]])

# y = torch.ones(x.shape)
y = torch.ones((1, 3, 4, 4))

x, y

(tensor([[[[ 1.7625,  0.1708, -0.1781,  0.9177],
           [ 0.1090,  0.9607,  0.5122, -0.6102],
           [-0.8960,  0.6751, -1.7255, -1.3522],
           [ 0.4167, -0.9028, -1.2607, -0.7607]],
 
          [[-0.2336,  0.3692, -1.5689, -1.3419],
           [ 0.3439, -0.5653,  0.1135, -1.2895],
           [-0.2301,  1.0384, -0.0396,  1.1815],
           [ 0.3712, -1.6379,  0.5888,  0.9896]],
 
          [[-0.3111, -0.4093,  0.9849,  0.5334],
           [ 0.2598,  0.4593,  0.0198, -0.8385],
           [-0.4612, -0.2769,  0.6749, -2.8046],
           [-1.1521,  0.2051, -0.1032,  0.9607]]]]),
 tensor([[[[1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.]],
 
          [[1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.]],
 
          [[1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.],
           [1., 1., 1., 1.]]]]))

In [3]:
conv = nn.Conv2d(3, 4, 2)
expected = conv(y)
expected

tensor([[[[-0.1194, -0.1194, -0.1194],
          [-0.1194, -0.1194, -0.1194],
          [-0.1194, -0.1194, -0.1194]],

         [[ 0.6854,  0.6854,  0.6854],
          [ 0.6854,  0.6854,  0.6854],
          [ 0.6854,  0.6854,  0.6854]],

         [[ 0.1693,  0.1693,  0.1693],
          [ 0.1693,  0.1693,  0.1693],
          [ 0.1693,  0.1693,  0.1693]],

         [[ 0.1380,  0.1380,  0.1380],
          [ 0.1380,  0.1380,  0.1380],
          [ 0.1380,  0.1380,  0.1380]]]])

In [4]:
# torch.nn.Unfold(kernel_size=2)(x)

kernel_size = (2, 2)
out_channels = 4

# Output of convolution as a matrix product
unfolded = torch.nn.functional.unfold(y, kernel_size=kernel_size)
# unfolded

wxb = conv.weight.view(out_channels, -1) @ unfolded + conv.bias.view(1, -1, 1)
actual = wxb.view(1, out_channels, y.shape[2] - kernel_size[0] + 1, y.shape[3] - kernel_size[1]+ 1)
torch.testing.assert_allclose(actual, expected)

### Testing our Conv2d Layer 

Forward pass

In [20]:
# testing our conv2d layer
my_conv = Conv2d(3, 4, 2)
x2 = my_conv.forward(x)
x2.shape

torch.Size([1, 4, 3, 3])

In [21]:
torch.testing.assert_allclose(my_conv.forward(x), torch.nn.functional.conv2d(x, my_conv.weight, my_conv.bias))

Backward pass

In [14]:
x2.shape

torch.Size([1, 4, 3, 3])

In [19]:
(x2.view(3, -1) @ w.view(4, -1).t()).shape

torch.Size([3, 4])

In [16]:
w = my_conv.weight
w.shape

torch.Size([4, 3, 2, 2])

In [ ]:
in_unfolded = unfold(x, kernel_size=2)
out_unfolded = my_conv.weight.view(4, -1) @ in_unfolded + my_conv.bias.view(1, -1, 1)
ouput = out_unfolded.view(x.shape[0], self.output_channels, tensor.shape[2]-self.kernel_size[0]+1, tensor.shape[3]-self.kernel_size[1]+1)

In [164]:
my_conv.backward(x2)

RuntimeError: The size of tensor a (3) must match the size of tensor b (4) at non-singleton dimension 1

### Testing our TransposeConv2d layer

Forward pass

In [11]:
my_t_conv = TransposeConv2d(3, 5, 2)
y_expected = my_t_conv.forward(x)
y_expected.shape

torch.Size([1, 5, 5, 5])

In [12]:
torch.testing.assert_allclose(my_t_conv.forward(x), torch.nn.functional.conv_transpose2d(x, my_t_conv.weight, my_t_conv.bias))

Backward pass

### Other tests

In [5]:
conv.weight.view(out_channels, -1).size()

torch.Size([4, 12])

In [118]:
x.size()

torch.Size([1, 3, 4, 4])

In [112]:
unfolded.size()

torch.Size([1, 12, 9])

In [115]:

wxb.size()

torch.Size([1, 4, 9])

In [81]:
# relu = ReLU()
f=Sigmoid()

f.forward(x)

tensor([[0.5000, 0.7311, 0.5000],
        [0.8808, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.7311]])

In [82]:
f.backward(y)

tensor([[0.5000, 0.2689, 0.5000],
        [0.1192, 0.5000, 0.5000],
        [0.5000, 0.5000, 0.2689]])